In [1]:
import pandas_market_calendars as mcal
import talib as ta
import yfinance as yf

#from google.colab import files
#uploaded = files.upload()
import io
import requests
import numpy as np  
import pandas as pd 
#from pandas.io.json import json_normalize
from datetime import datetime, timedelta

#import sklearn
#print(sklearn.__version__)
import matplotlib.pyplot as plt
#import scipy as sp 
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#from sklearn.ensemble import RandomForestRegressor
#from sklearn import metrics
#from sklearn.preprocessing import PolynomialFeatures
#from sklearn.feature_selection import SelectKBest
#from sklearn.feature_selection import f_regression
#from sklearn.model_selection import GridSearchCV
#from sklearn.naive_bayes import BernoulliNB
import sklearn.pipeline as skpipeline
#from sklearn.feature_selection import RFECV
#from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
#from sklearn.linear_model import LogisticRegression
#from sklearn.svm import LinearSVC
#from sklearn.feature_selection import SelectFromModel
#from sklearn.svm import SVC
from sklearn.decomposition import PCA, IncrementalPCA, KernelPCA
#from sklearn.model_selection import StratifiedKFold
#from sklearn.pipeline import FeatureUnion
#from sklearn.neural_network import MLPClassifier
#from sklearn.naive_bayes import GaussianNB
#from sklearn.base import BaseEstimator, TransformerMixin
#from sklearn.preprocessing import MultiLabelBinarizer
#from skmultilearn.problem_transform import BinaryRelevance
#from sklearn.preprocessing import LabelBinarizer
#from sklearn.multiclass import OneVsRestClassifier
#from sklearn.model_selection import cross_val_score
import pickle
import joblib
#from joblib import dump, load
#from sklearn.model_selection import ShuffleSplit
#import dask.dataframe as dd
#from dask_ml import model_selection
#from sklearn.datasets import make_classification
#import xgboost as xgb
import math
#from alpha_vantage.techindicators import TechIndicators

import random
from tempfile import mkdtemp
from shutil import rmtree
import gc

In [2]:
# Momentum-based RSI metrics 

# Set window for momentum/change period & number of periods to calculate RSI
#periods = [(2,6),(5,7),(10,8)]

def period_rsi(df, periods, *metrics):
  for metric in metrics:
    for day, ago in periods:
      df['{}{}'.format('rsi_mom',day)] = df[metric]/df.groupby('stock')[metric].shift(day)-1
      for a in range(ago+1):
        df['{}{}{}'.format('rsimom_pos', day, a)] = np.where(df.groupby('stock')['{}{}'.format('rsi_mom',day)].shift((day)*a)>=0, 
                                                            df.groupby('stock')['{}{}'.format('rsi_mom',day)].shift((day)*a), np.NaN)
        df['{}{}{}'.format('rsimom_neg', day, a)] = np.where(df.groupby('stock')['{}{}'.format('rsi_mom',day)].shift((day)*a)<=0, 
                                                            df.groupby('stock')['{}{}'.format('rsi_mom',day)].shift((day)*a), np.NaN)
      df['{}{}{}'.format('rsipos',day,ago)] = df.filter(regex=('{}{}{}'.format('rsimom_pos',day,'.*'))).mean(axis=1)
      df['{}{}{}'.format('rsineg',day,ago)] = -df.filter(regex=('{}{}{}'.format('rsimom_neg',day,'.*'))).mean(axis=1)
      df['{}{}{}'.format(metric, 'rsi_mom', day)] = np.where(df['{}{}{}'.format('rsineg',day,ago)].isna(), 100, 
                                                        np.where(df['{}{}{}'.format('rsipos',day,ago)].isna(), 0, 
                                                                  np.where(df['{}{}{}'.format('rsineg',day,ago)] == df['{}{}{}'.format('rsipos',day,ago)], 50, 
                                                                      100 - 100/(1+df['{}{}{}'.format('rsipos',day,ago)]/df['{}{}{}'.format('rsineg',day,ago)]))))
      df = df.drop(['{}{}{}'.format('rsipos',day,ago), '{}{}{}'.format('rsineg',day,ago)], axis=1)
      for a in range(ago+1):
        df = df.drop(['{}{}{}'.format('rsimom_pos', day, a), '{}{}{}'.format('rsimom_neg', day, a)], axis=1)
      df = df.drop(['{}{}'.format('rsi_mom',day)], axis=1)
  return df

def period_rsidiff(df, periods, *metrics):
  for metric in metrics:
    for day, ago in periods:
      df['{}{}'.format('rsi_mom',day)] = df[metric] - df.groupby('stock')[metric].shift(day)
      for a in range(ago+1):
        df['{}{}{}'.format('rsimom_pos', day, a)] = np.where(df.groupby('stock')['{}{}'.format('rsi_mom',day)].shift((day)*a)>=0, 
                                                            df.groupby('stock')['{}{}'.format('rsi_mom',day)].shift((day)*a), np.NaN)
        df['{}{}{}'.format('rsimom_neg', day, a)] = np.where(df.groupby('stock')['{}{}'.format('rsi_mom',day)].shift((day)*a)<=0, 
                                                            df.groupby('stock')['{}{}'.format('rsi_mom',day)].shift((day)*a), np.NaN)
      df['{}{}{}'.format('rsipos',day,ago)] = df.filter(regex=('{}{}{}'.format('rsimom_pos',day,'.*'))).mean(axis=1)
      df['{}{}{}'.format('rsineg',day,ago)] = -df.filter(regex=('{}{}{}'.format('rsimom_neg',day,'.*'))).mean(axis=1)
      df['{}{}{}'.format(metric, 'rsi_mom', day)] = np.where(df['{}{}{}'.format('rsineg',day,ago)].isna(), 100, 
                                                        np.where(df['{}{}{}'.format('rsipos',day,ago)].isna(), 0, 
                                                                  np.where(df['{}{}{}'.format('rsineg',day,ago)] == df['{}{}{}'.format('rsipos',day,ago)], 50, 
                                                                           100 - 100/(1+df['{}{}{}'.format('rsipos',day,ago)]/df['{}{}{}'.format('rsineg',day,ago)]))))
      df = df.drop(['{}{}{}'.format('rsipos',day,ago), '{}{}{}'.format('rsineg',day,ago)], axis=1)
      for a in range(ago+1):
        df = df.drop(['{}{}{}'.format('rsimom_pos', day, a), '{}{}{}'.format('rsimom_neg', day, a)], axis=1)
      df = df.drop(['{}{}'.format('rsi_mom',day)], axis=1)
  return df

In [3]:
# MACD Functions for Stock & SPY: calculates MACD metrics ((exp ma short/exp ma long)-1, macd, rsi of macd change)

# set windows for rsi function applied to MACD
#rsi_ranges = [6,18]
def macd(df, short, longg, signal, rsi_ranges, periods):
    df['{}{}{}{}'.format('macdd', signal, short, longg)]=(df.groupby('stock')['c'].rolling(short, win_type='triang').mean().reset_index(level=0, drop=True) \
    /df.groupby('stock')['c'].rolling(longg, win_type='triang').mean().reset_index(level=0, drop=True)) - 1
    
    df['{}{}{}{}'.format('macd', signal, short, longg)]= (df.groupby('stock')['c'].rolling(short, win_type='triang').mean().reset_index(level=0, drop=True) \
    - df.groupby('stock')['c'].rolling(longg, win_type='triang').mean().reset_index(level=0, drop=True))

    df['{}{}{}{}'.format('macd', signal, short, longg)]=df['{}{}{}{}'.format('macd', signal, short, longg)] \
    - df.groupby('stock')['{}{}{}{}'.format('macd', signal, short, longg)].rolling(signal, win_type='triang').mean().reset_index(level=0, drop=True)

    #df['{}{}{}{}{}'.format('macd', signal, short, longg, 'ch')] = df.groupby('stock')['{}{}{}{}'.format('macd', signal, short, longg)].diff()/df.groupby('stock')['{}{}{}{}'.format('macd', signal, short, longg)].shift(1)
    #df=consec(df, '{}{}{}{}'.format('macd', signal, short, longg), '{}{}{}{}{}'.format('macd', signal, short, longg, 'ch'))
    #df=metric_rsi(df, ['{}{}{}{}{}'.format('macd', signal, short, longg, 'ch')], rsi_ranges #[5,11,21,32,42,53]
    #              )
    df=period_rsidiff(df, periods, '{}{}{}{}'.format('macd', signal, short, longg))

    #df=df.drop(['{}{}{}{}{}'.format('macd', signal, short, longg, 'ch')], axis=1)

    return df

def spymacd(df, short, longg, signal, rsi_ranges, periods):
    df['{}{}{}{}'.format('spymacdd', signal, short, longg)]=(df.groupby('stock')['spy_c'].rolling(short, win_type='triang').mean().reset_index(level=0, drop=True) \
    /df.groupby('stock')['spy_c'].rolling(longg, win_type='triang').mean().reset_index(level=0, drop=True)) - 1
    
    df['{}{}{}{}'.format('spymacd', signal, short, longg)]= (df.groupby('stock')['spy_c'].rolling(short, win_type='triang').mean().reset_index(level=0, drop=True) \
    - df.groupby('stock')['spy_c'].rolling(longg, win_type='triang').mean().reset_index(level=0, drop=True))

    df['{}{}{}{}'.format('spymacd', signal, short, longg)]=df['{}{}{}{}'.format('spymacd', signal, short, longg)] \
    - df.groupby('stock')['{}{}{}{}'.format('spymacd', signal, short, longg)].rolling(signal, win_type='triang').mean().reset_index(level=0, drop=True) 
    
    #df['{}{}{}{}{}'.format('spymacd', signal, short, longg, 'ch')] = df.groupby('stock')['{}{}{}{}'.format('spymacd', signal, short, longg)].diff()/df.groupby('stock')['{}{}{}{}'.format('spymacd', signal, short, longg)].shift(1)
    #df=consec(df, '{}{}{}{}'.format('spymacd', signal, short, longg), '{}{}{}{}{}'.format('spymacd', signal, short, longg, 'ch'))
    #df=metric_rsi(df, ['{}{}{}{}{}'.format('spymacd', signal, short, longg, 'ch')], rsi_ranges #[5,11,21,32,42,53]
    #              )
    df=period_rsidiff(df, periods, '{}{}{}{}'.format('spymacd', signal, short, longg))

    #df=df.drop(['{}{}{}{}{}'.format('spymacd', signal, short, longg, 'ch')], axis=1)

    return df

In [4]:
# Drawdown Functions: days from max/min drawdown & difference from current price over window 'days'
def drawdown(df, *days):
  df['rollmax'] = df.groupby('stock')['c'].cummax().reset_index(level=0, drop=True)
  df['counter'] = np.where(df['rollmax']==df['c'],1,0)
  df['swings'] = df.groupby('stock')['counter'].cumsum()
  df['rollmax_dddays'] = df.groupby(['stock','swings']).cumcount()
  #df['rollmax'] = df['c']/df['rollmax']-1 
  #df['rollmax'] = df['c'] - df['rollmax']

  df['rollmin'] = df.groupby('stock')['c'].cummin().reset_index(level=0, drop=True)
  df['counter'] = np.where(df['rollmin']==df['c'],1,0)
  df['swings'] = df.groupby('stock')['counter'].cumsum()
  df['rollmin_dddays'] = df.groupby(['stock','swings']).cumcount()
  #df['rollmin'] = df['c']/df['rollmin']-1 
  #df['rollmin'] = df['c'] - df['rollmin']

  df = df.drop(['counter','swings'], axis=1)
  
  for day in days:
    df['{}{}'.format('rollmax',day)] = df.groupby('stock')['c'].rolling(day).max().reset_index(level=0, drop=True)
    df['counter'] = np.where(df['{}{}'.format('rollmax',day)]==df['c'],1,0)
    df['swings'] = df.groupby('stock')['counter'].cumsum()
    df['{}{}'.format('rollmax_dddays', day)] = df.groupby(['stock','swings']).cumcount()
    #df['{}{}'.format('rollmax',day)] = df['c']/df['{}{}'.format('rollmax',day)]-1 
    #df['{}{}'.format('rollmax',day)] = df['c'] - df['{}{}'.format('rollmax',day)]

    df['{}{}'.format('rollmin',day)] = df.groupby('stock')['c'].rolling(day).min().reset_index(level=0, drop=True)
    df['counter'] = np.where(df['{}{}'.format('rollmin',day)]==df['c'],1,0)
    df['swings'] = df.groupby('stock')['counter'].cumsum()
    df['{}{}'.format('rollmin_dddays', day)] = df.groupby(['stock','swings']).cumcount()
    #df['{}{}'.format('rollmin',day)] = df['c']/df['{}{}'.format('rollmin',day)]-1 
    #df['{}{}'.format('rollmin',day)] = df['c'] - df['{}{}'.format('rollmin',day)]

    df = df.drop(['counter','swings'], axis=1)
    
    df['{}{}'.format('aroon_bull', day)] = 100*(day - df['{}{}'.format('rollmax_dddays', day)])/day
    df['{}{}'.format('aroon_bear', day)] = 100*(day - df['{}{}'.format('rollmin_dddays', day)])/day
  
  return df

In [5]:
# Oscillator Function: oscilaitor calculation over long & short anchor windows 'days'
def oscillator(df, *days):
  for (k,d) in days:
    df['{}{}'.format('osc', k)]=100*(df['c']-df.groupby('stock')['c'].rolling(k).min().reset_index(level=0, drop=True)) \
    /(df.groupby('stock')['c'].rolling(k).max().reset_index(level=0, drop=True)-df.groupby('stock')['c'].rolling(k).min().reset_index(level=0, drop=True))

    df['{}{}{}{}'.format('osc', k, d, 'sma')]=df.groupby('stock')['{}{}'.format('osc', k)].rolling(d).mean().reset_index(level=0, drop=True)
    df['{}{}{}{}'.format('osc',k,d,'diff')]=df['{}{}'.format('osc', k)]-df['{}{}{}{}'.format('osc', k, d, 'sma')]
    df = df.drop(['{}{}'.format('osc', k), '{}{}{}{}'.format('osc', k, d, 'sma')], axis=1)
  return df

In [6]:
# Correlation Function for Beta Calculations
def correl(x, day):
    return pd.DataFrame(x['daily_return'].rolling(day).corr(x['spy_return']))

# Beta Calculation

def beta(df, beta_days):
  for day in beta_days: #13, 21, 34, 55
    df['{}{}'.format(day,'day_corr')] = df.groupby('stock')[['daily_return','spy_return']].apply(correl, day).shift(day).reset_index(level=0, drop=True)
    df['{}{}'.format(day,'day_stdcalc')] = df.groupby('stock')['daily_return'].rolling(day).std().shift(day).reset_index(level=0, drop=True) \
    /df.groupby('stock')['spy_return'].rolling(day).std().shift(day).reset_index(level=0, drop=True)
    df['{}{}'.format('beta_',day)] = df['{}{}'.format(day,'day_stdcalc')] * df['{}{}'.format(day,'day_corr')] 
    df=df.drop(['{}{}'.format(day,'day_corr'),'{}{}'.format(day,'day_stdcalc')], axis=1)
  return df

In [7]:
# Daily Change Function in percent 
def daily_change(df, **kwargs):
  for key, value in kwargs.items():
    df[key]=df.groupby('stock')[value].pct_change()
    #df[key]=df[value] - df.groupby('stock')[value].shift(1)
  return df 

In [8]:
# Date-range metrics
def preprocess(df):

# Next day's open & closing price
  df['cplus1'] = df.groupby('stock')['c'].shift(-1)
  df['cplus2'] = df.groupby('stock')['cplus1'].shift(-1)
  df['cplus3'] = df.groupby('stock')['cplus2'].shift(-1)
  df['cplus4'] = df.groupby('stock')['cplus3'].shift(-1)
  df['cplus5'] = df.groupby('stock')['cplus4'].shift(-1)
  df['cplus6'] = df.groupby('stock')['cplus5'].shift(-1)
  df['oplus1'] = df.groupby('stock')['o'].shift(-1)

# Daily closing & volume change 
  df=daily_change(df, daily_return='c', daily_volch='v')


# Isolate SPY
  spy_df = df[(df['stock'] == 'SPY')][['daily_return', 't', 'c']]
  spy_df=spy_df.rename(columns = {'daily_return':'spy_return', 'c':'spy_c'})
  df=pd.merge(df, spy_df, on='t')
  df = df[(df['stock'] != 'SPY')].sort_values(['stock', 't']).reset_index(drop=True) 

# On-Balance Volume Calculation
  #df['pvol'] = df['c']*df['v']
  #df['volsign'] = np.where(df['daily_return']>0, df['v'], -df['v'])
  #df['obv'] = df.groupby('stock')['volsign'].cumsum()
  #df['obvch'] = df['obv']/df.groupby('stock')['obv'].shift(1)-1
  #df = df.drop(['volsign'#, 'obv'
  #              ], axis=1) 

# Price x On-Balance Volume
  df['pobv'] = df['c']*df['obv']
  #df['pvolch'] = df['pvol']/df.groupby('stock')['pvol'].shift(1)-1
  df['pobvch'] = df['pobv']/df.groupby('stock')['pobv'].shift(1)-1

  return df


In [9]:
# Date-range metrics
def main(df, day_ranges, periods, rsi_ranges):

# metrics created for each window below for each stock and for SPY: momentum, moving avg deviation, standard deviation (volatility), change in mad, 
  for day in day_ranges:
    df['{}{}'.format('momentum',day)] = df['c']/df.groupby('stock')['c'].shift(day)-1
    df['{}{}'.format('obv_mom',day)] = df['obv']/df.groupby('stock')['obv'].shift(day)-1
    df['{}{}'.format('pobv_mom',day)] = df['pobv']/df.groupby('stock')['pobv'].shift(day)-1
    #df['{}{}'.format('mad',day)] = df['c']/df.groupby('stock')['c'].rolling(day).mean().reset_index(level=0, drop=True) - 1
    df['{}{}'.format('mad',day)] = df['c'] - df.groupby('stock')['c'].rolling(day).mean().reset_index(level=0, drop=True)
    #df['{}{}'.format('volma',day)] = df['v'] - df.groupby('stock')['v'].rolling(day).mean().reset_index(level=0, drop=True)
    #df['{}{}'.format('volma',day)] = df['v'] - df.groupby('stock')['v'].rolling(day).mean().reset_index(level=0, drop=True)
    #df['{}{}'.format('obvma',day)] = df['obv']/df.groupby('stock')['obv'].rolling(day).mean().reset_index(level=0, drop=True) - 1
    df['{}{}'.format('obvma',day)] = df['obv'] - df.groupby('stock')['obv'].rolling(day).mean().reset_index(level=0, drop=True)
    #df['{}{}'.format('pvolma',day)] = df['pvol'] - df.groupby('stock')['pvol'].rolling(day).mean().reset_index(level=0, drop=True)
    #df['{}{}'.format('pobvma',day)] = df['pobv']/df.groupby('stock')['pobv'].rolling(day).mean().reset_index(level=0, drop=True) - 1
    df['{}{}'.format('pobvma',day)] = df['pobv'] - df.groupby('stock')['pobv'].rolling(day).mean().reset_index(level=0, drop=True)
    #df['{}{}'.format('bolltop_',day)] = df['c'] /(df['{}{}'.format('mad_',day)] - 2*df.groupby('stock')['c'].rolling(day).std().reset_index(level=0, drop=True)) - 1
    df['{}{}'.format('std',day)] = df.groupby('stock')['c'].rolling(day).std().reset_index(level=0, drop=True)/df['c']
    df['{}{}'.format('vol_std',day)] = df.groupby('stock')['v'].rolling(day).std().reset_index(level=0, drop=True)/df['v']
    df['{}{}'.format('pobv_std',day)] = df.groupby('stock')['pobv'].rolling(day).std().reset_index(level=0, drop=True)/df['pobv']

    df['{}{}'.format('spymomentum',day)] = df['spy_c']/df.groupby('stock')['spy_c'].shift(day)-1
    #df['{}{}'.format('spymad',day)] = df['spy_c']/df.groupby('stock')['spy_c'].rolling(day).mean().reset_index(level=0, drop=True) - 1
    df['{}{}'.format('spymad',day)] = df['spy_c'] - df.groupby('stock')['spy_c'].rolling(day).mean().reset_index(level=0, drop=True)
    #df['{}{}'.format('bolltop_',day)] = df['c'] /(df['{}{}'.format('mad_',day)] - 2*df.groupby('stock')['c'].rolling(day).std().reset_index(level=0, drop=True)) - 1
    df['{}{}'.format('spystd',day)] = df.groupby('stock')['spy_c'].rolling(day).std().reset_index(level=0, drop=True)/df['spy_c']
    
    df['{}{}'.format('madch',day)] = df.groupby('stock')['{}{}'.format('mad', day)].diff()#/df.groupby('stock')['{}{}'.format('mad', day)].shift(day)
    df['{}{}'.format('spymadch',day)] = df.groupby('stock')['{}{}'.format('spymad', day)].diff()#/df.groupby('stock')['{}{}'.format('spymad', day)].shift(day)
    df['{}{}'.format('obvmach',day)] = df.groupby('stock')['{}{}'.format('obvma', day)].diff()#/df.groupby('stock')['{}{}'.format('obvma', day)].shift(day)
    df['{}{}'.format('pobvmach',day)] = df.groupby('stock')['{}{}'.format('pobvma', day)].diff()#/df.groupby('stock')['{}{}'.format('pobvma', day)].shift(day)

  #for day in day_ranges:
    #df=consec(df, '{}{}'.format('momentum',day), #'{}{}'.format('obv_mom',day), 
    #          '{}{}'.format('pobv_mom',day),  #'{}{}'.format('pobvma',day), 
    #          '{}{}'.format('spymomentum',day)#, '{}{}'.format('mad',day)
    #          )
  #for day in day_ranges_slim:
    #df=metric_rsi(df, ['{}{}'.format('momentum',day), '{}{}'.format('pobv_mom',day), '{}{}'.format('spymomentum',day), '{}{}'.format('madch',day), '{}{}'.format('spymadch',day),  
    #                   '{}{}'.format('obvmach',day), 
    #                   '{}{}'.format('pobvmach',day)
    #                  ], rsi_ranges #[5,11,21,32,42,53]
    #              )

  #for day in day_ranges:
  #  df=consec(df, '{}{}'.format('mad',day), '{}{}'.format('spymad',day), '{}{}'.format('madch',day), '{}{}'.format('spymadch',day)) 
  #for day in day_ranges_slim:
  #  df=metric_rsi(df, ['{}{}'.format('madch',day), '{}{}'.format('spymadch',day),  
  #                    '{}{}'.format('obvmach',day), '{}{}'.format('pobvmach',day)], rsi_ranges #[5,11,21,32,42,53]
  #                )
    
  return df
  #df=df.drop(['{}{}'.format('madch',day)], axis=1) 
  #df=df.drop(['{}{}'.format('spymadch',day)], axis=1) 


In [10]:
sector = pd.read_csv("https://raw.githubusercontent.com/wangjason11/sandbox/master/retire/universe_500m.csv")

In [11]:
stocks = sector['Symbol'].tolist()
stocks.append('SPY')
#stocks = ['BRK-B', 'BH-A', 'TAP-A']
data = {}
L = []

In [12]:
# Ending_date set to today (-24 hours because colab on UTC)
ending_date = datetime.today() # - timedelta(hours=24) 
end_date = ending_date.strftime('%Y-%m-%d')
#end_date = end_date + "T09:30:00-04:00" #formatting needed for api call
end_date

'2021-03-17'

In [13]:
nyse = mcal.get_calendar('NYSE')

# Trading days since 2014
date_range = nyse.schedule(start_date = '2014-10-01', end_date = ending_date.strftime('%Y-%m-%d'))
                          #end_date='2019-10-06')

data_limit = 200 # trading days
date_range=date_range.loc[::-1].reset_index()
ending=date_range.iloc[data_limit][0].strftime('%Y-%m-%d')
#ending = ending + "T09:30:00-04:00" #formatting needed for api call
print(ending)

2020-06-01


In [14]:
prediction_date = (datetime.today() #- timedelta(hours=24)
                  ).strftime('%Y-%m-%d')

In [ ]:
for stock in stocks:
    print(stock)
    for attempt in range(100):
        print(attempt)
        try:
            data[stock] = yf.download(tickers = stock, period = "max" , interval = "1d")
            break
        except KeyError:
            x = stock
            L.append(x)
            break
        except ConnectionError:
            print('sleeping')
            time.sleep(attempt*5)
        except AssertionError:
            x = stock
            L.append(x)
            break

AAPL
0
[*********************100%***********************]  1 of 1 completed
MSFT
0
[*********************100%***********************]  1 of 1 completed
AMZN
0
[*********************100%***********************]  1 of 1 completed
GOOG
0
[*********************100%***********************]  1 of 1 completed
GOOGL
0
[*********************100%***********************]  1 of 1 completed
TSLA
0
[*********************100%***********************]  1 of 1 completed
FB
0
[*********************100%***********************]  1 of 1 completed
BABA
0
[*********************100%***********************]  1 of 1 completed
TSM
0
[*********************100%***********************]  1 of 1 completed
BRK-B
0
[*********************100%***********************]  1 of 1 completed
V
0
[*********************100%***********************]  1 of 1 completed
JNJ
0
[*********************100%***********************]  1 of 1 completed
JPM
0
[*********************100%***********************]  1 of 1 completed
WMT
0
[***********

[*********************100%***********************]  1 of 1 completed
DE
0
[*********************100%***********************]  1 of 1 completed
DEO
0
[*********************100%***********************]  1 of 1 completed
CVS
0
[*********************100%***********************]  1 of 1 completed
MELI
0
[*********************100%***********************]  1 of 1 completed
TGT
0
[*********************100%***********************]  1 of 1 completed
ISRG
0
[*********************100%***********************]  1 of 1 completed
LMT
0
[*********************100%***********************]  1 of 1 completed
SYK
0
[*********************100%***********************]  1 of 1 completed
SNAP
0
[*********************100%***********************]  1 of 1 completed
VALE
0
[*********************100%***********************]  1 of 1 completed
GSK
0
[*********************100%***********************]  1 of 1 completed
BKNG
0
[*********************100%***********************]  1 of 1 completed
BTI
0
[********************

[*********************100%***********************]  1 of 1 completed
GIS
0
[*********************100%***********************]  1 of 1 completed
ALXN
0
[*********************100%***********************]  1 of 1 completed
ANSS
0
[*********************100%***********************]  1 of 1 completed
BGNE
0
[*********************100%***********************]  1 of 1 completed
U
0
[*********************100%***********************]  1 of 1 completed
CNC
0
[*********************100%***********************]  1 of 1 completed
DDOG
0
[*********************100%***********************]  1 of 1 completed
BF-A
0
[*********************100%***********************]  1 of 1 completed
PCAR
0
[*********************100%***********************]  1 of 1 completed
PSX
0
[*********************100%***********************]  1 of 1 completed
CS
0
[*********************100%***********************]  1 of 1 completed
KMI
0
[*********************100%***********************]  1 of 1 completed
ZBH
0
[*********************

[*********************100%***********************]  1 of 1 completed
FOX
0
[*********************100%***********************]  1 of 1 completed
FUTU
0
[*********************100%***********************]  1 of 1 completed
INCY
0
[*********************100%***********************]  1 of 1 completed
TRMB
0
[*********************100%***********************]  1 of 1 completed
HPE
0
[*********************100%***********************]  1 of 1 completed
ROL
0
[*********************100%***********************]  1 of 1 completed
ASX
0
[*********************100%***********************]  1 of 1 completed
HES
0
[*********************100%***********************]  1 of 1 completed
ALNY
0
[*********************100%***********************]  1 of 1 completed
IR
0
[*********************100%***********************]  1 of 1 completed
ZEN
0
[*********************100%***********************]  1 of 1 completed
BEPC
0
[*********************100%***********************]  1 of 1 completed
LYV
0
[********************

Exception in thread Thread-655:
Traceback (most recent call last):
  File "C:\Users\wangj\anaconda3\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\wangj\anaconda3\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\wangj\anaconda3\lib\site-packages\multitasking\__init__.py", line 102, in _run_via_pool
    return callee(*args, **kwargs)
  File "C:\Users\wangj\anaconda3\lib\site-packages\yfinance\multi.py", line 167, in _download_one_threaded
    actions, period, interval, prepost, proxy, rounding)
  File "C:\Users\wangj\anaconda3\lib\site-packages\yfinance\multi.py", line 182, in _download_one
    rounding=rounding, many=True)
  File "C:\Users\wangj\anaconda3\lib\site-packages\yfinance\base.py", line 153, in history
    raise RuntimeError("*** YAHOO! FINANCE IS CURRENTLY DOWN! ***\n"
RuntimeError: *** YAHOO! FINANCE IS CURRENTLY DOWN! ***
Our engineers are working quickly to resolve the issue. Thank you for

In [ ]:
for key, value in data.items():
    value['stock'] = key
    value['obv'] = ta.OBV(value['Close'], value['Volume'])
    value['adosc3_10'] = ta.ADOSC(value['High'], value['Low'], value['Close'], value['Volume'], fastperiod=3, slowperiod=10)
    value['adosc5_20'] = ta.ADOSC(value['High'], value['Low'], value['Close'], value['Volume'], fastperiod=5, slowperiod=20)
    value['adx7'] = ta.ADX(value['High'],  value['Low'], value['Close'], timeperiod=7)
    value['adx15'] = ta.ADX(value['High'],  value['Low'], value['Close'], timeperiod=15)
    value['adx40'] = ta.ADX(value['High'],  value['Low'], value['Close'], timeperiod=40)
    value['willr11'] = ta.WILLR(value['High'], value['Low'], value['Close'], timeperiod=11)
    value['willr21'] = ta.WILLR(value['High'], value['Low'], value['Close'], timeperiod=21)
    value['willr45'] = ta.WILLR(value['High'], value['Low'], value['Close'], timeperiod=45)
    value['willr84'] = ta.WILLR(value['High'], value['Low'], value['Close'], timeperiod=84)
    value['kamad7'] = value['Close'] - ta.KAMA(value['Close'], timeperiod=7) 
    value['kamad15'] = value['Close'] - ta.KAMA(value['Close'], timeperiod=15) 
    value['kamad31'] = value['Close'] - ta.KAMA(value['Close'], timeperiod=31) 
    value['emad7'] = value['Close'] - ta.EMA(value['Close'], timeperiod=7) 
    value['emad15'] = value['Close'] - ta.EMA(value['Close'], timeperiod=15) 
    value['emad31'] = value['Close'] - ta.EMA(value['Close'], timeperiod=31) 

In [ ]:
#for key, value in data.items():
#    value['stock'] = key
#    value['dol_v'] = value['Close']*value['Volume']

In [ ]:
df = pd.concat([df for df in data.values()])

In [ ]:
df.reset_index(inplace=True)
df = df.rename(columns = {'Date':'t', 'Open': 'o', 'High': 'h', 'Low': 'l', 'Close': 'c', 'Volume': 'v', 'Adj Close': 'adj_c'})
df['stock'] = df['stock'].astype('category')

In [ ]:
#df['dolv_125']=df.groupby('stock')['dol_v'].rolling(125, min_periods=1).mean().reset_index(level=0, drop=True)
#df=df[df['t']=='2021-02-12']
#df=pd.merge(df, sector, left_on='stock', right_on='Symbol').reset_index(drop=True)

In [ ]:
#df.to_csv('universe_selection.csv', sep=',', mode='w', index=False)

In [ ]:
del data, stocks, L, sector
gc.collect()

In [ ]:
cols = df.select_dtypes(exclude=['object', 'datetime', 'uint8', 'int64', 'float32', 'category']).columns

df[cols] = df[cols].astype('float32')

In [ ]:
del cols
gc.collect()

In [ ]:
df['t'] = pd.to_datetime(df['t'], unit='s').astype('datetime64[D]')
df=df[df['t'] >= ending]
df=df.sort_values(['stock', 't']).reset_index(drop=True)

In [ ]:
#cols = df.columns

#for col in cols:
#    print(col, len(df[col]), len(df[col]) - df[col].count())

In [ ]:
#df[df['o'].isna()]

In [ ]:
df=preprocess(df)

In [ ]:
# set day windows for day_ranges below for metric calculations
day_ranges = [3,7,15,31] #3,5,11,21,42
day_ranges_slim = [3,7,15] #3,5,11,21,42
rsi_ranges = [14]
periods = [(2,10),(5,8),(10,6)
           ]
# can optionally re-set windows for rsi calculations later on

In [ ]:
df=main(df, day_ranges, periods, rsi_ranges)

In [ ]:
for day in day_ranges_slim:
    df=period_rsidiff(df, periods, '{}{}'.format('obvma',day), '{}{}'.format('pobvma',day), '{}{}'.format('mad',day), '{}{}'.format('spymad',day),
              #'{}{}'.format('momentum',day), '{}{}'.format('pobv_mom',day), '{}{}'.format('spymomentum',day), 
              '{}{}'.format('madch',day), '{}{}'.format('spymadch',day),  
              '{}{}'.format('obvmach',day), 
              '{}{}'.format('pobvmach',day),
              '{}{}'.format('pobv_std',day)
              )

In [ ]:
# set windows for beta function
beta_days = [11,30,60]
df=beta(df, beta_days)

In [ ]:
# Additional Metrics from functions defined above
df=oscillator(df, (13,2), #(11,3), 
           (35,6), 
           #(42,7), 
           #(84,10)
           (65,10)
           ) 

In [ ]:
# Additional Metrics from functions defined above
df=drawdown(df, 11,21,45,84#,84
         )

In [ ]:
df=macd(df, 7, 11, 21, rsi_ranges, periods)

In [ ]:
df=macd(df, 21, 28, 56, rsi_ranges, periods)
#macd(df, 38, 50, 100)

In [ ]:
df=spymacd(df, 7, 11, 21, rsi_ranges, periods)

In [ ]:
df=spymacd(df, 21, 28, 56, rsi_ranges, periods)
#spymacd(df, 38, 50, 100)

In [ ]:
#df=consec(df, 'daily_return', 'spy_return', 'pobvch')

# can optionally re-set windows for rsi calculations applied to MACD
#rsi_ranges = [5,11,21,32,42,53]
#df=metric_rsi(df, ['daily_return', 'obvch', 'spy_return', 'pobvch'], rsi_ranges #[5,11,21,32,42,53]
#              )
df=period_rsi(df, periods, 'c', 'obv', 'pobv', 'spy_c'#, 'pvol'
              )

In [ ]:
df['kamadch7'] = df.groupby('stock')['kamad7'].diff()#/df.groupby('stock')['{}{}'.format('mad', day)].shift(day)
df['kamadch15'] = df.groupby('stock')['kamad15'].diff()#/df.groupby('stock')['{}{}'.format('spymad', day)].shift(day)
df['kamadch31'] = df.groupby('stock')['kamad31'].diff()#/df.groupby('stock')['{}{}'.format('obvma', day)].shift(day)


In [ ]:
df['emadch7'] = df.groupby('stock')['emad7'].diff()#/df.groupby('stock')['{}{}'.format('pobvma', day)].shift(day)
df['emadch15'] = df.groupby('stock')['emad15'].diff()#/df.groupby('stock')['{}{}'.format('pobvma', day)].shift(day)
df['emadch31'] = df.groupby('stock')['emad31'].diff()#/df.groupby('stock')['{}{}'.format('pobvma', day)].shift(day)


In [ ]:
df['adxch7'] = df.groupby('stock')['adx7'].diff()#/df.groupby('stock')['{}{}'.format('pobvma', day)].shift(day)
df['adxch15'] = df.groupby('stock')['adx15'].diff()#/df.groupby('stock')['{}{}'.format('pobvma', day)].shift(day)
df['adxch40'] = df.groupby('stock')['adx40'].diff()#/df.groupby('stock')['{}{}'.format('pobvma', day)].shift(day)


In [ ]:
df=period_rsidiff(df, periods, 'kamad7', 'kamad15', 'kamad31', 'emad7', 'emad15', 'emad31', 'adx7', 'adx15', 'adx40'
              )

In [ ]:
df=period_rsidiff(df, periods, 'kamadch7', 'kamadch15', 'kamadch31', 'emadch7', 'emadch15', 'emadch31', 'adxch7', 'adxch15', 'adxch40'
              )

In [ ]:
#"""
# Create metrics of existing metrics from b1 & b2 days ago

#df.columns.tolist()
b1=6
b2=21
#b3=33
backago=df.drop(['stock', 't', 'o', #'obvch', 'pobvch', 
                 #'c', 'v', 
                 'cplus1','oplus1',#'pvol',
                 'daily_return', #'daily_volch', 
                 'spy_return', 'cplus2', 'h', 'l', 'adj_c', 'cplus2', 'cplus3', 'cplus4', 'cplus5', 'cplus6'#, 'spy_c'
                 ], axis=1).drop(df.filter(regex='{}{}{}'.format('.*','ch','[0-9]{0,2}$')).columns, axis=1).columns.tolist()


#df.drop(df.filter(regex='my_expression').columns,axis=1)

#dff.filter(regex=('{}{}'.format('buy','.*'))).describe()

In [ ]:
for metric in backago:
  df['{}{}{}'.format(metric, '_b', b1)]=df.groupby('stock', observed=True)[metric].shift(b1).reset_index(drop=True).astype('float32')
  #df['{}{}{}'.format(metric, '_b', b3)]=df.groupby('stock', observed=True)[metric].shift(b3).reset_index(drop=True).astype('float32')
#"""

In [ ]:
for metric in backago:
  df['{}{}{}'.format(metric, '_b', b2)]=df.groupby('stock', observed=True)[metric].shift(b2).reset_index(drop=True).astype('float32')
  #df['{}{}{}'.format(metric, '_b', b3)]=df.groupby('stock', observed=True)[metric].shift(b3).reset_index(drop=True).astype('float32')
#"""

In [ ]:
del backago
gc.collect()

In [ ]:
# Bitcoin & VIX data
bitcoin_data = pd.read_csv("https://data.bitcoinity.org/export_data.csv?currency=USD&data_type=price_volume&r=day&t=lb&timespan=all&vu=curr", header=0)
#bitcoin_data = pd.read_csv("bitcoinity_data (6).csv", header=0)
bitcoin_data.columns=['date', 'price', 'volume']
bitcoin_data['date'] = bitcoin_data['date'].str[:10].astype('datetime64[D]')
spy_sector = pd.read_csv("https://raw.githubusercontent.com/wangjason11/sandbox/master/retire/universe_500m.csv")
vix = pd.read_csv("https://ww2.cboe.com/publish/scheduledtask/mktdata/datahouse/vixcurrent.csv", skiprows=2)
vix.columns=['date', 'vix_open', 'vix_high', 'vix_low', 'vix_close']
vix['date'] = vix['date'].astype('datetime64[D]')

In [ ]:
# REMEMBER TO CHANGE BTC END DATE WHEN MAKING PREDICTIONS
bitcoin_end = ending_date #datetime.today() - timedelta(hours=24) 
bitcoin_end = bitcoin_end.strftime('%Y%m%d')

# Add BTC data
bdf=bitcoin_data[bitcoin_data['date'].isin(pd.date_range(start='20141001', end=bitcoin_end, freq='B').strftime('%Y%m%d'))].reset_index(drop=True)
bdf['cap'] = bdf['price']*bdf['volume']
bdf = bdf[['date', 'cap']]


In [ ]:
for day in day_ranges:
    bdf['{}{}'.format('btcch', day)] = bdf['cap']/bdf['cap'].shift(day)-1


In [ ]:
#print(bdf.iloc[0:50,:])
bdf=bdf.rename(columns = {'date':'btc_date'})
dff=pd.merge(df, bdf, left_on='t', right_on='btc_date').reset_index(drop=True)


In [ ]:
# Add Sector data
spy_sector = spy_sector[['Symbol', 'Sector', 'Industry', 'Sub-Industry', 'market_cap']]
spy_sector = spy_sector.rename(columns={"Symbol": "symbol", "Sector": "sector", "Industry": "industry", "Sub-Industry": "sub_industry"})
spy_sector['symbol'] = spy_sector['symbol'].astype('category')
spy_sector['sector'] = spy_sector['sector'].astype('category')
spy_sector['industry'] = spy_sector['industry'].astype('category')
spy_sector['sub_industry'] = spy_sector['sub_industry'].astype('category')
spy_sector['market_cap'] = spy_sector['market_cap'].astype('category')

dff=pd.merge(dff, spy_sector, left_on='stock', right_on='symbol').reset_index(drop=True)

In [ ]:
# Add VIX data
vix = vix[['date', 'vix_close']]

for day in day_ranges:
    vix['{}{}'.format('vixch', day)] = vix['vix_close']/vix['vix_close'].shift(day)-1


In [ ]:
vix=vix.rename(columns = {'date':'vix_date'})
dff=pd.merge(dff, vix, left_on='t', right_on='vix_date').sort_values(['stock', 't']).reset_index(drop=True)

#dff.describe()

In [ ]:
del bitcoin_data, spy_sector, vix, bdf, df
gc.collect()

In [ ]:
# convert numeric data type for faster processing

cols = dff.select_dtypes(exclude=['object', 'datetime', 'uint8', 'int64', 'float32', 'category']).columns

dff[cols] = dff[cols].astype('float32')

In [ ]:
del cols
gc.collect()

In [ ]:
#cols = dff.columns

#for col in cols:
#    print(col, len(dff[col]), len(dff[col]) - dff[col].count())

In [ ]:
#"""
def returns(dff, *days):
  for day in days:
    dff['{}{}'.format('return_', day)]=dff['{}{}'.format('cplus', (day+1))]/dff['cplus1']-1
    dff['{}{}'.format('buy_', day)]=np.where(dff['{}{}'.format('return_', day)]>(day+0)/100, 1, 0)
  return dff
#dff['nextd_return']=dff['nextd_return'].replace([np.inf, -np.inf, np.nan], -1.1)
#"""

In [ ]:
dff = returns(dff, 1)


In [ ]:
dff = returns(dff, 2)


In [ ]:
dff = returns(dff, 3)


In [ ]:
dff = returns(dff, 4)


In [ ]:
dff = returns(dff, 5)


In [ ]:
dff['buy_sum'] = np.where(dff['buy_1'] + dff['buy_2'] + dff['buy_3'] + dff['buy_4'] + dff['buy_5'] > 0, 1, 0)

In [ ]:
#"""
def returns2(dff, *days):
  for day in days:
    dff['{}{}{}'.format('return_', day, '_2')]=dff['{}{}'.format('cplus', (day+1))]/dff['cplus1']-1
    dff['{}{}{}'.format('buy_', day, '_2')]=np.where(dff['{}{}'.format('return_', day)]>(day+1)/100, 1, 0)
  return dff
#dff['nextd_return']=dff['nextd_return'].replace([np.inf, -np.inf, np.nan], -1.1)
#"""

In [ ]:
dff = returns2(dff, 1)


In [ ]:
dff = returns2(dff, 2)


In [ ]:
dff = returns2(dff, 3)


In [ ]:
dff = returns2(dff, 4)


In [ ]:
dff = returns2(dff, 5)


In [ ]:
dff['buy_sum_2'] = np.where(dff['buy_1_2'] + dff['buy_2_2'] + dff['buy_3_2'] + dff['buy_4_2'] + dff['buy_5_2'] > 0, 1, 0)

In [ ]:
#"""
#dff[['buy_3d', 'buy_4d', 'buy_5d',
#     #'sell_8d', 'buy_8d', 'sell_11d',  'buy_11d', #'sell_13d',  'buy_13d', 'sell_16d',  'buy_16d', 'sell_18d', 'buy_18d', 'sell_21d',  'buy_21d', 
#     'buy', 'buy_expand'#, 'buyall', 'buy8'
     #, 'buy11+', 'buy11-21', 'buy11-32', 'buy11&32', 'buy11&42', 'buy16+', 'buy16-32', 'buy16&32', 'buy16&42',
     #'buy21+', 'buy21-32', 'buy21&42',  'buy32+'
#    ]].describe()

dff.filter(regex=('{}{}'.format('buy','.*'))).describe()
#"""
#dff[['buy', 'buy_5d', 'sell_5d']].describe()

In [ ]:
#dff['stock'].nunique()

In [ ]:
# Drop non-features

dff=dff.drop([#'c', 'v', 
              'cplus1', #'spy_c', #'spy_return', 
              'vix_date', 'btc_date', 'symbol', 'oplus1', #'o', #'obvch',
              'cap', 'cplus2', 'cplus3', 'cplus4', 'cplus5', 'cplus6'#, 'vix_close'#, 'sector'
             ], axis=1)


In [ ]:
dff[['stock', 'sector', 'industry', 'sub_industry', 'market_cap']] = dff[['stock', 'sector', 'industry', 'sub_industry', 'market_cap']].astype(str)

In [ ]:
cutoff = ending_date - timedelta(days=10) # can be set to other values longer or shorter
#predict=dff[dff.t > cutoff.strftime('%Y-%m-%d')]
final=dff[dff['t'] > cutoff.strftime('%Y-%m-%d')]
#final=final[final['t'] > '2017-10-18']


In [ ]:
final['stock'].nunique()

In [ ]:
#final['t'].nunique()

In [ ]:
#final=final.dropna(subset=['return_1', 'return_2', 'return_3', 'return_4', 'return_5'
#                      , 'return_1_2', 'return_2_2', 'return_3_2', 'return_4_2', 'return_5_2', 'stock', 't'])

In [ ]:
#final['stock'].nunique()

In [ ]:
#final['t'].nunique()

In [ ]:
# Remove NULLs

#dff['nextd_return']=dff['nextd_return'].replace([np.inf, -np.inf, np.nan], -1.1)
final['return_1']=final['return_1'].replace([np.inf, -np.inf, np.nan], -1.1)
final['return_2']=final['return_2'].replace([np.inf, -np.inf, np.nan], -1.1)
final['return_3']=final['return_3'].replace([np.inf, -np.inf, np.nan], -1.1)
final['return_4']=final['return_4'].replace([np.inf, -np.inf, np.nan], -1.1)
final['return_5']=final['return_5'].replace([np.inf, -np.inf, np.nan], -1.1)


In [ ]:
# Remove NULLs

#dff['nextd_return']=dff['nextd_return'].replace([np.inf, -np.inf, np.nan], -1.1)
final['return_1_2']=final['return_1_2'].replace([np.inf, -np.inf, np.nan], -1.1)
final['return_2_2']=final['return_2_2'].replace([np.inf, -np.inf, np.nan], -1.1)
final['return_3_2']=final['return_3_2'].replace([np.inf, -np.inf, np.nan], -1.1)
final['return_4_2']=final['return_4_2'].replace([np.inf, -np.inf, np.nan], -1.1)
final['return_5_2']=final['return_5_2'].replace([np.inf, -np.inf, np.nan], -1.1)


In [ ]:
#dff['nulls'] = np.where(dff.isnull().any(axis=1), 'delete', 'keep')
#dff = dff[dff.nulls == 'keep']
#dff[dff.nulls == 'delete']

In [ ]:
#dff=dff.drop(['nulls'], axis=1)

In [ ]:
#dff=dff.replace([np.inf, -np.inf], np.nan)
#for to_replace in [np.inf, -np.inf]:
#    dff = dff.mask(dff == to_replace, np.nan) 


In [ ]:
#dff=dff.dropna()

In [ ]:
#dff['nulls'] = np.where(dff.isnull().any(axis=1), 'delete', 'keep')
#dff = dff[dff.nulls == 'keep']
#dff=dff.drop(['nulls'], axis=1)

In [ ]:
final=final.sort_values(['stock', 't']).reset_index(drop=True)

In [ ]:
final.filter(regex=('{}{}'.format('buy','.*'))).describe()

In [ ]:
#dff.stock.nunique()

In [ ]:
#dff.shape

In [ ]:
#cols = dff.columns

#for col in cols:
#    print(col, len(dff[col]), len(dff[col]) - dff[col].count())

In [ ]:
#"""
#prediction_date = (datetime.today() #- timedelta(hours=24)
#                  ).strftime('%Y-%m-%d')
#file_name="resig_opt_{}.csv".format(prediction_date)
#final.to_csv(file_name, sep=',', mode='w', index=False)
#from google.colab import files  
#files.download(file_name)
#"""

In [ ]:
sectors = pd.read_csv("https://raw.githubusercontent.com/wangjason11/sandbox/master/retire/universe_500m.csv")
sectors = sectors[['Symbol', 'Market Capitalization', 'dolv_125']]

In [ ]:
#trim=pd.merge(final, sectors, left_on='stock', right_on='Symbol').reset_index(drop=True)

In [ ]:
#trim['Market Capitalization'] = trim['Market Capitalization'].astype('int64')

In [ ]:
#trim=trim[(trim['Market Capitalization']>=600000000) & (trim['dolv_125']>=3000000)]

In [ ]:
#trim=trim.drop(['Market Capitalization', 'Symbol', 'dolv_125'], axis = 1) 

In [ ]:
#len(final)

In [ ]:
#len(trim)

In [ ]:
#trim['stock'].nunique()

In [ ]:
#final['stock'].nunique()

In [ ]:
#"""
#prediction_date = (datetime.today() #- timedelta(hours=24)
#                  ).strftime('%Y-%m-%d')
#file_name="resig_optrim_{}.csv".format(prediction_date)
#trim.to_csv(file_name, sep=',', mode='w', index=False)
#from google.colab import files  
#files.download(file_name)
#"""

In [ ]:
trim2=pd.merge(final, sectors, left_on='stock', right_on='Symbol').reset_index(drop=True)
trim2['Market Capitalization'] = trim2['Market Capitalization'].astype('int64')
trim2=trim2[(trim2['Market Capitalization']>=700000000) & (trim2['dolv_125']>=3500000)]
trim2=trim2.drop(['Symbol'], axis = 1) 
trim2['stock'].nunique()

In [ ]:
#"""
#prediction_date = (datetime.today() #- timedelta(hours=24)
#                  ).strftime('%Y-%m-%d')
file_name="resig_optrim2_predict_{}.csv".format(prediction_date)
trim2.to_csv(file_name, sep=',', mode='w', index=False)
#from google.colab import files  
#files.download(file_name)
#"""

In [ ]:
#trim.filter(regex=('{}{}'.format('buy','.*'))).describe()

In [ ]:
trim2.filter(regex=('{}{}'.format('buy','.*'))).describe()

## ------ END -------